In [1]:
import sys

sys.path.append("../Lib")

from MRRecordUtil import *

In [2]:
data_type, postfix = '腹痛', '人机'
keys, json_data = load_data(r'../data/%s/汇总结果_%s.json' % (data_type, postfix), '../data/%s/labeled_ind_%s.txt' % (data_type, postfix))

keys length: 158, json data length: 158


In [4]:
for record in json_data:
    print('%s	%s	%s' % (
        get_json_value(record, ['医保编号']),
        get_json_value(record, ['入院时间']),
        get_json_value(record, ['出院记录', '出院时间'])
    ))

10262279	20220121	20220216
10269015	20220212	20220218
10269175	20220212	20220307
10269310	20220213	
10284732	20220321	20220401
10291585	20220406	20220418
10292908	20220410	20220414
10307840	20220514	20220608
10312747	20220524	20220607
10327229	20220622	20220630
20074613	20220130	20220208
20084843	20220423	20220501
20087548	20220513	20220524
10261670	20220119	20220204
10264842	20220131	20220214
10265236	20220204	20220214
10267976	20220209	20220218
10273667	20220222	20220226
10274389	20220223	20220305
10275284	20220227	20220303
10277568	20220305	20220314
10278542	20220308	
10287200	20220328	20220401
10290860	20220406	20220408
10291394	20220406	20220409
10265017	20220202	20220215
10274832	20220225	20220310
10281448	20220314	20220329
10299754	20220425	20220506
10301442	20220429	20220518
10318303	20220605	20220624
10264928	20220201	
10265471	20220205	20220211
10265796	20220206	20220210
10271543	20220217	20220311
10275990	20220301	20220315
10285085	20220322	20220326
10290175	20220403	2022041

### 加入人工补充和门诊外院的影像学数据

In [3]:
xlsx_path = r'D:\项目资料\病历辅助诊断\特征抽取\数据\影像学（门诊+院外+手工）.xlsx'
xlsx_data = load_sheet_arr_dict(xlsx_path, 'Sheet3')

In [4]:
for record, adata in zip(json_data, xlsx_data):
    print(record['医保编号'], adata['医保编号'])
    for key in ['人工数据1', '人工数据2', '人工数据3', '人工数据4', '人工数据5', '人工数据6', '门诊外院1', '门诊外院2']:
        if adata[key] == '':
            continue
            
        arr = adata[key].split(',')
        print(arr[0], arr[1])
        
        if arr[1] == '超声':
            val_str = '%s,%s' % ('US', ','.join(arr[2:]))
            if '超声' not in record:
                record['超声'] = []
                
            cs_record = None
            for item in record['超声']:
                if item['日期'] == arr[0]:
                    cs_record = item
                    break
                    
            if cs_record is None:
                record['超声'].append({
                    '日期': arr[0],
                    '数据': [val_str]
                })
            else:
                cs_record['数据'].append(val_str)
        elif arr[1] in ['CT', 'MR', 'DR']:
            val_str = ','.join(arr[1:])
            if '放射' not in record:
                record['放射'] = []
                
            fs_record = None
            for item in record['放射']:
                if item['日期'] == arr[0]:
                    fs_record = item
                    break
                    
            if fs_record is None:
                record['放射'].append({
                    '日期': arr[0],
                    '数据': [val_str]
                })
            else:
                fs_record['数据'].append(val_str)
            
        else:
            print(arr)
            
with open(r'../data/%s/汇总结果_%s_append.json' % (data_type, postfix), "w") as f:
        f.write(json.dumps(json_data, indent=1, separators=(',', ':'), ensure_ascii=False))

10262279 10262279
20220216 DR
20220216 CT
10269015 10269015
20220218 DR
20220212 CT
20220212 CT
10269175 10269175
20220307 DR
20220307 CT
20220212 CT
10269310 10269310
20220318 DR
20220318 CT
10269310 CT
20220213 CT
20220213 CT
10284732 10284732
20220401 DR
20220401 CT
10284732 CT
20220401 CT
10291585 10291585
20220418 DR
20220406 CT
10292908 10292908
20220414 DR
20220414 CT
10292908 CT
10307840 10307840
20220608 DR
20220608 CT
10312747 10312747
20220607 DR
20220607 CT
10312747 CT
20220523 CT
10327229 10327229
20220630 DR
10327229 CT
20220630 CT
20074613 20074613
20220208 DR
20220208 CT
20074613 CT
20220208 CT
20220208 CT
20084843 20084843
20220501 DR
20220501 CT
20084843 CT
20220501 CT
20220501 CT
20087548 20087548
20220524 DR
20220524 CT
10261670 10261670
20220119 CT
10264842 10264842
20220214 DR
20220214 CT
20220214 CT
10265236 10265236
20220214 DR
10265236 CT
20220214 CT
10267976 10267976
20220218 DR
10267976 CT
20220218 CT
20220218 CT
10273667 10273667
20220226 DR
20220226 CT
2022

### 替换原始的实验室数据，加入选取以及人工补充的实验室数据

In [6]:
xlsx_path2 = r'D:\项目资料\病历辅助诊断\特征抽取\数据\人机大赛实验室数据.xlsx'
xlsx_data_lab = load_sheet_arr_dict(xlsx_path2, 'Sheet1')

In [7]:
data_type, postfix = '腹痛', '人机'
keys, json_data = load_data(r'../data/%s/汇总结果_%s.json' % (data_type, postfix), '../data/%s/labeled_ind_%s.txt' % (data_type, postfix))

keys length: 158, json data length: 158


In [8]:
for record, adata in zip(json_data, xlsx_data_lab):
    print(record['医保编号'], adata['医保编号'])
    ## 清空检验数据
    record['检验'] = []
    for key in ['中性粒细胞%', '白细胞', 'C反应蛋白', '降钙素原', '脂肪酶', '淀粉酶', 'β-绒毛膜促性腺激素', '总胆红素', \
                '天冬氨酸氨基转移酶', '丙氨酸氨基转移酶', '碱性磷酸酶', 'γ-谷氨酰转移酶', '血沉', '红细胞', '尿白细胞', '尿红细胞']:
        if adata[key] == '':
            continue
        
        # excel 一格有多行数据。
        item_lines = adata[key].split('\n')
            
        for item_line in item_lines:
            arr = item_line.split(',')
            date, val_str = arr[0], ','.join(arr[1:])
            
            # 根据日期找记录
            date_record = None
            for json_item in record['检验']:
                if json_item['日期'] == date:
                    date_record = json_item
                    break
                    
            # 如果没有此日期数据
            if date_record is None:
                record['检验'].append({
                    '日期': date,
                    '数据': [val_str]
                })
            else:
                date_record['数据'].append(val_str)
                
    # 按照日期排序
    record['检验'] = sorted(record['检验'], key=lambda x: x['日期'])
            
with open(r'../data/%s/汇总结果_%s_append.json' % (data_type, postfix), "w") as f:
        f.write(json.dumps(json_data, indent=1, separators=(',', ':'), ensure_ascii=False))

10262279 10262279
10269015 10269015
10269175 10269175
10269310 10269310
10284732 10284732
10291585 10291585
10292908 10292908
10307840 10307840
10312747 10312747
10327229 10327229
20074613 20074613
20084843 20084843
20087548 20087548
10261670 10261670
10264842 10264842
10265236 10265236
10267976 10267976
10273667 10273667
10274389 10274389
10275284 10275284
10277568 10277568
10278542 10278542
10287200 10287200
10290860 10290860
10291394 10291394
10265017 10265017
10274832 10274832
10281448 10281448
10299754 10299754
10301442 10301442
10318303 10318303
10264928 10264928
10265471 10265471
10265796 10265796
10271543 10271543
10275990 10275990
10285085 10285085
10290175 10290175
10293601 10293601
10297101 10297101
10302914 10302914
10303505 10303505
10311977 10311977
10313245 10313245
10314902 10314902
10315400 10315400
10318010 10318010
10319397 10319397
10321439 10321439
10322698 10322698
20076153 20076153
20076427 20076427
20083449 20083449
10265319 10265319
10272676 10272676
10281528 1